## Summary
Teasing emailing campaign, only to UNIQUE emails

## Process
- Load list of eligible beneficiaries from 2025
- JSON extraction for the allocataire field
- Data mapping and deletion of unusable data
- URL generation containing the code
- Output 3 files
  - Indirect beneficiaries
    - Family with one children
    - Familiy with more than one children
  - Direct beneficiairy

## Target audience
- Jeunes : 14 ans au 31 decembre 2025
- Jeunes : 18 ans au 16 septembre 2025
- => donc jeunes nés entre le 31.12.2025 et 16.09.2007

- AAH : >= 20 ans

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
import json
import time
from data.utils.emailing_utils import format_allocataire_benef_names_in_place

load_dotenv()

start_time = time.time()

benef_2024_pathfile = os.environ['BENEF_2024_TEASING_PARQUET_PATHFILE']

output_one_children= os.environ['CAMPAIGN_TEASING_FAMILY_ONE_CHILDREN_2025']
output_multiple_children = os.environ['CAMPAIGN_TEASING_FAMILY_MULTIPLE_CHILDREN_2025']
output_direct_beneficiaries = os.environ['CAMPAIGN_TEASING_DIRECT_BENEFICIARIES_2025']

In [ ]:
df_main = pd.read_parquet(benef_2024_pathfile)

In [ ]:
df_json_normalized = pd.json_normalize(df_main['allocataire'].apply(json.loads))
df_json_normalized = df_json_normalized.add_prefix('allocataire_')
df_main.index = pd.RangeIndex(start=0, stop=len(df_main), step=1)
df_unwrapped_alloc = pd.merge(df_main, df_json_normalized, left_index=True, right_index=True)

print(f"Number of beneficiaries : {len(df_unwrapped_alloc)}")

In [ ]:
# Remove RGPD lines
rgpd_emails = os.environ['RGPD_EMAILS_TO_EXCLUDE_PATHFILE']
rgpd_codes = os.environ['RGPD_CODES_TO_EXCLUDE_PATHFILE']

df_rgpd_emails = pd.read_csv(rgpd_emails)
df_rgpd_codes = pd.read_csv(rgpd_codes)

print(f"Number of beneficiaries before excluding RGPD lines : {len(df_unwrapped_alloc)} ")

df_unwrapped_alloc = df_unwrapped_alloc[~df_unwrapped_alloc['id'].isin(df_rgpd_codes['id'])]
df_unwrapped_alloc = df_unwrapped_alloc[~df_unwrapped_alloc['allocataire_courriel'].str.lower().isin(df_rgpd_emails['email'].str.lower())]

print(f"Number of beneficiaries after excluding RGPD lines : {len(df_unwrapped_alloc)} ")

In [ ]:
# Users that have email
mask_not_existing_email = df_unwrapped_alloc['allocataire_courriel'].apply(lambda x: pd.isna(x) or x == '')
df_unwrapped_alloc = df_unwrapped_alloc[~mask_not_existing_email]

print(f"Number of beneficiaries with email : {len(df_unwrapped_alloc)}")

In [ ]:
# Only take benef who have been updated in 2024 and beyond
# df_unwrapped_alloc['updated_at'] = pd.to_datetime(df_unwrapped_alloc['updated_at'], format='ISO8601', errors='coerce')
df_unwrapped_alloc['updated_at'] = pd.to_datetime(df_unwrapped_alloc['updated_at'].apply(lambda v: v[:10]), format='%Y-%m-%d')
df_unwrapped_alloc = df_unwrapped_alloc[df_unwrapped_alloc['updated_at'].dt.year > 2023]

In [ ]:
# Exclude boursiers and invalidated benef
df_unwrapped_alloc = df_unwrapped_alloc[df_unwrapped_alloc['refuser'] == False]
df_unwrapped_alloc = df_unwrapped_alloc[df_unwrapped_alloc['a_valider'] == False]
df_unwrapped_alloc = df_unwrapped_alloc[df_unwrapped_alloc['situation'].str.lower() != 'boursier']

In [ ]:
len(df_unwrapped_alloc)

In [ ]:
from datetime import datetime

# Take only people between 14 years old and 18 years old
start_date = datetime(2007, 9, 16).date()
end_date = datetime(2011, 12, 31).date()

df_unwrapped_alloc['date_naissance'] = pd.to_datetime(df_unwrapped_alloc['date_naissance'], format='ISO8601', errors='coerce')

mask_14_to_18_years_old = (df_unwrapped_alloc['date_naissance'].dt.date >= start_date) & (df_unwrapped_alloc['date_naissance'].dt.date <= end_date)
df_14_to_18_years_old = df_unwrapped_alloc.loc[mask_14_to_18_years_old]

In [ ]:
print(f"{len(df_unwrapped_alloc)} original benef length")
print(f"{len(df_14_to_18_years_old)} benef between 14 and 18 years old")

In [ ]:
column_mapping = {
    'id': 'id',
    'genre': 'genre',
    'updated_at': 'updated_at',
    'organisme': 'organisme',
    'date_naissance': 'date_naissance',
    'situation': 'situation',
    'allocataire_courriel': 'email',
    'allocataire_qualite': 'allocataire_qualite',
    'allocataire_nom': 'allocataire_nom',
    'allocataire_prenom': 'allocataire_prenom',
    'allocataire_matricule': 'allocataire_matricule',
    'prenom': 'beneficiaire_prenom',
    'nom': 'beneficiaire_nom',
    'allocataire_telephone': 'telephone',
}

df_unwrapped_alloc.columns = df_unwrapped_alloc.columns.to_series().replace(column_mapping)

In [ ]:
# Format names
format_allocataire_benef_names_in_place(df_unwrapped_alloc)

In [ ]:
# Group indirect beneficiaries to distinguish family with 1 and famility with > 1 children
grouped_emails = df_unwrapped_alloc.groupby(['email']).size().reset_index(name="count")
single_families = grouped_emails[grouped_emails['count'] == 1]
families_more_than_one = grouped_emails[grouped_emails['count'] > 1]

In [ ]:
# Unique AAH taken from families with one beneficiary
mask_alloc_eq_benef = df_unwrapped_alloc['beneficiaire_prenom'].str.lower() == df_unwrapped_alloc['allocataire_prenom'].str.lower()

# todo: Update period once Julianne has the definitive dates
# AAH
# original period for AAH in old scripts for 2024
# start_date = datetime(1993, 9, 16).date() # 30-31 years old
# end_date = datetime(2008, 12, 31).date() # 16 years old

aah_start_date = datetime(1994, 9, 16).date() # 30 years old
aah_end_date = datetime(2005, 12, 31).date() # 20 years old

mask_aah = (df_unwrapped_alloc['date_naissance'].dt.date <= aah_start_date) & (df_unwrapped_alloc['date_naissance'].dt.date <= aah_end_date)
aah_df = df_unwrapped_alloc[mask_aah & mask_alloc_eq_benef]

print(f"{len(aah_df)} AAH benef")

unique_aah_df = aah_df[aah_df['email'].isin(single_families['email'])]

In [ ]:
# allocataire = bénéficiaire
mask_alloc_eq_benef = df_unwrapped_alloc['beneficiaire_prenom'].str.lower() == df_unwrapped_alloc['allocataire_prenom'].str.lower()

direct_benef = df_unwrapped_alloc.loc[df_unwrapped_alloc['email'].isin(single_families['email'])].loc[mask_alloc_eq_benef]
indirect_benef = df_unwrapped_alloc[~df_unwrapped_alloc['email'].isin(direct_benef['email'])]

In [ ]:
# Families with single kid
indirect_benef_single_families = df_unwrapped_alloc[df_unwrapped_alloc['email'].isin(single_families['email'])]
indirect_benef_single_families = indirect_benef_single_families[~indirect_benef_single_families['email'].isin(direct_benef['email'])]

In [ ]:
# Families with mulitple kids
indirect_benef_siblings_families = df_unwrapped_alloc[df_unwrapped_alloc['email'].isin(families_more_than_one['email'])]
indirect_benef_siblings_families = indirect_benef_siblings_families[~indirect_benef_siblings_families['email'].isin(direct_benef['email'])]
indirect_benef_siblings_families = indirect_benef_siblings_families[~indirect_benef_siblings_families['email'].isin(indirect_benef_single_families['email'])]
indirect_benef_siblings_families = indirect_benef_siblings_families.drop_duplicates(subset=['email']) # Only single email per family

In [ ]:
# Indirect benef should not have any AAH
assert(indirect_benef_single_families['email'].isin(unique_aah_df['email']).any() == False)
assert(indirect_benef_siblings_families['email'].isin(unique_aah_df['email']).any() == False)
assert(len(pd.merge(unique_aah_df, indirect_benef_siblings_families, on='email', how='inner')) == 0)
assert(len(pd.merge(unique_aah_df, indirect_benef_single_families, on='email', how='inner')) == 0)

In [ ]:
# Indirect benef should not contain any email from direct benef
assert(indirect_benef_single_families['email'].isin(direct_benef['email']).any() == False)

# Indirect benef from families with more than one children should not contain any email from single families and direct beneficiaries
assert(indirect_benef_siblings_families['email'].isin(direct_benef['email']).any() == False)
assert(indirect_benef_siblings_families['email'].isin(indirect_benef_single_families['email']).any() == False)

In [ ]:
print(f"{len(indirect_benef_single_families)} families with only one children")
print(f"{len(indirect_benef_siblings_families)} families with more than one children")
print(f"{len(unique_aah_df)} aah direct beneficiaries")
print(f"{len(indirect_benef_single_families) + len(indirect_benef_siblings_families)} families with one kid and more than one children")
print(f"Total : {len(indirect_benef_single_families) + len(indirect_benef_siblings_families) + len(unique_aah_df)}")

In [ ]:
columns_to_keep = [
    'id',
    'email',
    'beneficiaire_prenom',
    'beneficiaire_nom',
    'allocataire_nom',
    'allocataire_prenom',
    'date_naissance',
    'genre',
    'organisme',
    'situation'
    # 'allocataire_matricule',
    # 'organisme'
]

indirect_benef_single_families[columns_to_keep].to_csv(output_one_children, index=False)
indirect_benef_siblings_families[columns_to_keep].to_csv(output_multiple_children, index=False)
unique_aah_df[columns_to_keep].to_csv(output_direct_beneficiaries, index=False)

end_time = time.time()
print(f"Notebook executed in {end_time - start_time:.2f} seconds")